In [85]:
import pandas as pd                                       # procesamiento de datos
from datetime import timedelta                            # para incrementos de fechas
from oandapyV20 import API                                # conexion con broker OANDA
import oandapyV20.endpoints.instruments as instruments    # informacion de precios historicos
import pandas as pd
import plotly.graph_objs as go                            # objetos de imagenes para funcion principal
import plotly.io as pio                                   # renderizador para visualizar imagenes
import numpy as np                                        # funciones numericas
pio.renderers.default = "notebook"                        # render de imagenes para correr en script
import warnings
warnings.filterwarnings("ignore")
import math

In [86]:
def f_precios_masivos(p0_fini, p1_ffin, p2_gran, p3_inst, p4_oatk, p5_ginc):
    """
    Parameters
    ----------
    p0_fini : str : fecha inicial para descargar precios en formato str o pd.to_datetime
    p1_ffin : str : fecha final para descargar precios en formato str o pd.to_datetime
    p2_gran : str : M1, M5, M15, M30, H1, H4, H8, segun formato solicitado por OANDAV20 api
    p3_inst : str : nombre de instrumento, segun formato solicitado por OANDAV20 api
    p4_oatk : str : OANDAV20 API
    p5_ginc : int : cantidad de datos historicos por llamada, obligatorio < 5000
    Returns
    -------
    dc_precios : pd.DataFrame : Data Frame con precios TOHLC
    Debugging
    ---------
    p0_fini = pd.to_datetime("2019-01-01 00:00:00").tz_localize('GMT')
    p1_ffin = pd.to_datetime("2019-12-31 00:00:00").tz_localize('GMT')
    p2_gran = "M1"
    p3_inst = "USD_MXN"
    p4_oatk = Tu token
    p5_ginc = 4900
    """

    def f_datetime_range_fx(p0_start, p1_end, p2_inc, p3_delta):
        """
        Parameters
        ----------
        p0_start : str : fecha inicial
        p1_end : str : fecha final
        p2_inc : int : incremento en cantidad de elementos
        p3_delta : str : intervalo para medir elementos ('minutes', 'hours', 'days')
        Returns
        -------
        ls_result : list : lista con fechas intermedias a frequencia solicitada
        Debugging
        ---------
        p0_start = p0_fini
        p1_end = p1_ffin
        p2_inc = p5_ginc
        p3_delta = 'minutes'
        """

        ls_result = []
        nxt = p0_start

        while nxt <= p1_end:
            ls_result.append(nxt)
            if p3_delta == 'minutes':
                nxt += timedelta(minutes=p2_inc)
            elif p3_delta == 'hours':
                nxt += timedelta(hours=p2_inc)
            elif p3_delta == 'days':
                nxt += timedelta(days=p2_inc)

        return ls_result

    # inicializar api de OANDA

    api = API(access_token=p4_oatk)

    gn = {'S30': 30, 'S10': 10, 'S5': 5, 'M1': 60, 'M5': 60 * 5, 'M15': 60 * 15,
          'M30': 60 * 30, 'H1': 60 * 60, 'H4': 60 * 60 * 4, 'H8': 60 * 60 * 8,
          'D': 60 * 60 * 24, 'W': 60 * 60 * 24 * 7, 'M': 60 * 60 * 24 * 7 * 4}

    # -- para el caso donde con 1 peticion se cubran las 2 fechas
    if int((p1_ffin - p0_fini).total_seconds() / gn[p2_gran]) < 4990:

        # Fecha inicial y fecha final
        f1 = p0_fini.strftime('%Y-%m-%dT%H:%M:%S')
        f2 = p1_ffin.strftime('%Y-%m-%dT%H:%M:%S')

        # Parametros pra la peticion de precios
        params = {"granularity": p2_gran, "price": "M", "dailyAlignment": 16, "from": f1,
                  "to": f2}

        # Ejecutar la peticion de precios
        a1_req1 = instruments.InstrumentsCandles(instrument=p3_inst, params=params)
        a1_hist = api.request(a1_req1)

        # Para debuging
        # print(f1 + ' y ' + f2)
        lista = list()

        # Acomodar las llaves
        for i in range(len(a1_hist['candles']) - 1):
            lista.append({'TimeStamp': a1_hist['candles'][i]['time'],
                          'Open': a1_hist['candles'][i]['mid']['o'],
                          'High': a1_hist['candles'][i]['mid']['h'],
                          'Low': a1_hist['candles'][i]['mid']['l'],
                          'Close': a1_hist['candles'][i]['mid']['c']})

        # Acomodar en un data frame
        r_df_final = pd.DataFrame(lista)
        r_df_final = r_df_final[['TimeStamp', 'Open', 'High', 'Low', 'Close']]
        r_df_final['TimeStamp'] = pd.to_datetime(r_df_final['TimeStamp'])
        r_df_final['Open'] = pd.to_numeric(r_df_final['Open'], errors='coerce')
        r_df_final['High'] = pd.to_numeric(r_df_final['High'], errors='coerce')
        r_df_final['Low'] = pd.to_numeric(r_df_final['Low'], errors='coerce')
        r_df_final['Close'] = pd.to_numeric(r_df_final['Close'], errors='coerce')

        return r_df_final

    # -- para el caso donde se construyen fechas secuenciales
    else:

        # hacer series de fechas e iteraciones para pedir todos los precios
        fechas = f_datetime_range_fx(p0_start=p0_fini, p1_end=p1_ffin, p2_inc=p5_ginc,
                                     p3_delta='minutes')

        # Lista para ir guardando los data frames
        lista_df = list()

        for n_fecha in range(0, len(fechas) - 1):

            # Fecha inicial y fecha final
            f1 = fechas[n_fecha].strftime('%Y-%m-%dT%H:%M:%S')
            f2 = fechas[n_fecha + 1].strftime('%Y-%m-%dT%H:%M:%S')

            # Parametros pra la peticion de precios
            params = {"granularity": p2_gran, "price": "M", "dailyAlignment": 16, "from": f1,
                      "to": f2}

            # Ejecutar la peticion de precios
            a1_req1 = instruments.InstrumentsCandles(instrument=p3_inst, params=params)
            a1_hist = api.request(a1_req1)

            # Para debuging
            # print(f1 + ' y ' + f2)
            lista = list()

            # Acomodar las llaves
            for i in range(len(a1_hist['candles']) - 1):
                lista.append({'TimeStamp': a1_hist['candles'][i]['time'],
                              'Open': a1_hist['candles'][i]['mid']['o'],
                              'High': a1_hist['candles'][i]['mid']['h'],
                              'Low': a1_hist['candles'][i]['mid']['l'],
                              'Close': a1_hist['candles'][i]['mid']['c']})

            # Acomodar en un data frame
            pd_hist = pd.DataFrame(lista)
            pd_hist = pd_hist[['TimeStamp', 'Open', 'High', 'Low', 'Close']]
            pd_hist['TimeStamp'] = pd.to_datetime(pd_hist['TimeStamp'])

            # Ir guardando resultados en una lista
            lista_df.append(pd_hist)

        # Concatenar todas las listas
        r_df_final = pd.concat([lista_df[i] for i in range(0, len(lista_df))])

        # resetear index en dataframe resultante porque guarda los indices del dataframe pasado
        r_df_final = r_df_final.reset_index(drop=True)
        r_df_final['Open'] = pd.to_numeric(r_df_final['Open'], errors='coerce')
        r_df_final['High'] = pd.to_numeric(r_df_final['High'], errors='coerce')
        r_df_final['Low'] = pd.to_numeric(r_df_final['Low'], errors='coerce')
        r_df_final['Close'] = pd.to_numeric(r_df_final['Close'], errors='coerce')

        return r_df_final

In [92]:
oa_token = '03650f87204a34dcd5870adc0c7b5d7f-40b690db231ac0d6020b31c543f9ed82'
oa_in = "EUR_USD"  # Instrumento
oa_gn = "H1"       # Granularidad de velas (M1: Minuto, M5: 5 Minutos, M15: 15 Minutos)
fini = pd.to_datetime("2019-06-01 00:00:00").tz_localize('GMT')  # Fecha inicial
ffin = pd.to_datetime("2019-12-31 00:00:00").tz_localize('GMT')  # Fecha final

precios = f_precios_masivos(p0_fini=fini, p1_ffin=ffin, p2_gran=oa_gn,
                               p3_inst=oa_in, p4_oatk=oa_token, p5_ginc=4900)

In [93]:
precios['escenario'] = 0
precios_MayorAMenor=precios.sort_values('High',ascending=False)
precios_MenorAMAyor=precios.sort_values('Low',ascending=True)
PMayor=int(precios_MayorAMenor.iloc[0].name)
PMenor=int(precios_MenorAMAyor.iloc[0].name)

In [100]:
def g_velas(p0_de):
    """
    :param p0_de: data frame con datos a graficar
    :return fig:
    p0_de = datos_dd
    p1_pa = 'sell'
    datos_dd = pd.DataFrame({'timestamp': [], 'open': [], 'high': [], 'low': [], 'close': []}, index=[])
    """

    p0_de.columns = [list(p0_de.columns)[i].lower() for i in range(0, len(p0_de.columns))]

    fig = go.Figure(data=[go.Candlestick(x=p0_de['timestamp'],
                                         open=p0_de['open'], high=p0_de['high'],
                                         low=p0_de['low'], close=p0_de['close'])])

    fig.update_layout(margin=go.layout.Margin(l=50, r=50, b=20, t=50, pad=0),
                      title=dict(x=0.5, y=1, text='Precios Historicos OHLC'),
                      xaxis=dict(title_text='Hora del dia', rangeslider=dict(visible=False)),
                      yaxis=dict(title_text='Precio del EurUsd'))

    fig.layout.autosize = False
    fig.layout.width = 940
    fig.layout.height = 520
    #--------------------------------------------------------inicia primera linea de tendencia (PMayor)
    eme=[gh/1000000 for gh in range(0,-10000,-1)]
    equis=[i for i in range(len(precios))] #por que empezaria desd ela posicion del punto mayor osea 38
    be=precios['high'][PMayor]
    y=[]
    r=[]
    [y.append([]) for i in range(len(eme))] #para hacer muchas rectas con diferentes puntos


    for i in range(len(eme)):
        for j in range(len(equis)-PMayor):
            ye=eme[i]*equis[j]+be
            y[i].append(ye)
    for p in range(1,len(y)):
        for k in range(1,len(y[p])):
            if y[p][k]<=precios["high"][PMayor+k]:
                erre=p
                oo=k
                if k<PMenor:
                    break

        else:
            continue
        break  
                

    fig.add_shape(
        # Line Diagonal
        type="line",
        x0=precios['timestamp'][PMayor],
        y0=precios['high'][PMayor],
        x1=precios['timestamp'][oo+PMayor],
        y1=y[erre][oo],
        line=dict(
            color="Orange",
            width=4,
            dash="dot",
            )
    )
    #---------------------------------------termina primera linea de tendencia
    #---------------------------------------inicia segunda linea de tendencia(PMenor)
    eme=[gh/100000 for gh in range(0,10000,1)]
    equis=[i for i in range(len(precios))] #por que empezaria desd ela posicion del punto mayor osea 38
    be=precios['low'][PMenor]
    y=[]
    r=[]
    [y.append([]) for i in range(len(eme))] #para hacer muchas rectas con diferentes puntos


    for i in range(len(eme)):
        for j in range(len(equis)-PMenor):
            ye=eme[i]*equis[j]+be
            y[i].append(ye)    
    
    for p in range(1,len(y)):
        for k in range(1,len(y[p])):
            if y[p][k]>=precios["low"][PMenor+k]:
                erre=p
                oo=k
                break
        else:
            continue
        break 
                
     
    fig.add_shape(
        # Line Diagonal
        type="line",
        x0=precios['timestamp'][PMenor],
        y0=precios['low'][PMenor],
        x1=precios['timestamp'][(oo+PMenor)],
        y1=y[erre][oo],
        line=dict(
            color="Blue",
            width=4,
            dash="dot",
            )
    )
    fig.update_shapes(dict(xref='x', yref='y'))
    

    return fig


In [101]:
velas=g_velas(precios)

In [102]:
velas

In [84]:
#linea de tendencia bajista, desde el punto mayor
eme=[gh/1000000 for gh in range(0,-10000,-1)]
equis=[i for i in range(len(precios))] #por que empezaria desd ela posicion del punto mayor osea 38
be=precios['high'][PMayor]
y=[]
r=[]
[y.append([]) for i in range(len(eme))] #para hacer muchas rectas con diferentes puntos


for i in range(len(eme)):
    for j in range(len(equis)-PMayor):
        ye=eme[i]*equis[j]+be
        y[i].append(ye)
for p in range(1,len(y)):
    for k in range(1,len(y[p])):
        if y[p][k]<=precios["high"][PMayor+k]:
            erre=p
            oo=k
            if k<PMenor:
                break
                
    else:
        continue
    break 

In [83]:
#linea de tendencia alcista desde punto menor
eme=[gh/100000 for gh in range(0,10000,1)]
equis=[i for i in range(len(precios))] #por que empezaria desd ela posicion del punto mayor osea 38
be=precios['low'][PMenor]
y=[]
r=[]
[y.append([]) for i in range(len(eme))] #para hacer muchas rectas con diferentes puntos

for i in range(len(eme)):
    for j in range(len(equis)-PMenor):
        ye=eme[i]*equis[j]+be
        y[i].append(ye)    
    
for p in range(1,len(y)):
    for k in range(1,len(y[p])):
        if y[p][k]>=precios["low"][PMenor+k]:
            erre=p
            oo=k   
            break
    else:
        continue
    break 

In [251]:
precios['escenario'] = 0
precios_MayorAMenor=precios.sort_values('high',ascending=False)
precios_MenorAMAyor=precios.sort_values('low',ascending=True)

In [252]:
PMayor=int(precios_MayorAMenor.iloc[0].name)
PMenor=int(precios_MenorAMAyor.iloc[0].name)

El siguiente codigo es la tendencia asignada, tendencia que se le asiganará, dependiendo de los puntos mayores y menores de la grafica

- Si esta a la izquierda del punto mayor, sera alcista 1
- Si esta a la derecha del punto mayor, sera bajista -1

- Si esta a la izquierda del punto menor, sera bajista -1
- Si esta a la derecha del punto menor, sera alcista 1
- Si es punto mayor o menor sera 2

In [253]:
#alcista 1
#bajista -1
#pico mayor Y PICO MENOR 2
if PMayor<PMenor:
    NdfMay=precios.iloc[0:PMayor]
    Ndf=precios.iloc[PMayor:PMenor]
    NdfMen=precios.iloc[PMenor:-1]
    DF=NdfMay,Ndf,NdfMen
    
    DF[0]['escenario']=1
    DF[1]['escenario']=-1
    DF[2]['escenario']=1
    
    DF[1]['escenario'].iloc[0]=2
    DF[2]['escenario'].iloc[0]=-2
else:
    NdfMay=precios.iloc[0:PMenor]
    Ndf=precios.iloc[PMenor:PMayor]
    NdfMen=precios.iloc[PMayor:-1]
    DF=NdfMay,Ndf,NdfMen
    
    DF[0]['escenario']=-1
    DF[1]['escenario']=1
    DF[2]['escenario']=-1
    
    DF[1]['escenario'].iloc[0]=-2
    DF[2]['escenario'].iloc[0]=2

Esto se hace para ver si es bajista por denominación o alcista

- si el open es mayor que el close (esta en rojo en la grafica) es bajista
- si el close es mayor que el open (esta en verde en la grafica) es alcista

In [254]:
#alcista 1
#bajista -1
DF[0]['tendencia']=0
DF[1]['tendencia']=0
DF[2]['tendencia']=0

for i in range(len(DF)):
    for j in range(len(DF[i])):
        if DF[i]['open'].iloc[j]>DF[i]['close'].iloc[j]:
            DF[i]['tendencia'].iloc[j]=-1
        elif DF[i]['close'].iloc[j]>DF[i]['open'].iloc[j]:
            DF[i]['tendencia'].iloc[j]=1

A lo que siguiría igual la misma tendencia ya sea bajista o alcista lo dejaremos en 0 y a los cambios en cunato a la vela pasada lo pondremos como 1

In [255]:
#sin cambios=0
#cambios=1
DF[0]['cambios']=0
DF[1]['cambios']=0
DF[2]['cambios']=0
for i in range(len(DF)):
    for j in range(len(DF[i])):
        if DF[i]['tendencia'].iloc[j] != DF[i]['tendencia'].iloc[j-1]:
            DF[i]['cambios'].iloc[j]=1
        

Checar el punto medio de el close y el open para checar despues cambios porcentuales

In [256]:
DF[0]['pm']=0
DF[1]['pm']=0
DF[2]['pm']=0
for i in range(len(DF)):
    for j in range(len(DF[i])):
        DF[i]['pm'].iloc[j]=DF[i]['open'].iloc[j]/DF[i]['close'].iloc[j]
            

Hasta ahorita llevamos 4 parametros
- Escenario: de que lado de los puntos maximos y minimos esta en la grafica
- Tendencia:la tendencia que surgio en el dia, si sube o si baja
- Cambios: checa a ver si hizo lo contario que la vela pasada 
- Punto medio: da el punto medio entre el open y close para checar el movimeinto si es alto o bajo

eme=[1/math.tan(math.radians(i)) for i in range(180,270)]
eme=eme[::-1]
equis=[i for i in range(len(precios))] #por que empezaria desd ela posicion del punto mayor osea 38
be=precios['high'][PMayor]
y=[]
r=[]
[y.append([]) for i in range(len(eme))] #para hacer muchas rectas con diferentes puntos


for i in range(len(eme)):
    for j in range(len(equis)):
        ye=eme[i]*equis[j]+be
        y[i].append(ye)

                

for p in range(len(y)):
    for k in range(len(precios)-PMayor):
            if k>0:  
                if y[p][k]==precios["high"][PMayor+k] or y[p][k]<=precios["high"][PMayor+k]:
                    erre=y[p][k]
                    r.append(erre)